# Funções Auxiliares

## Métricas de desempenho tradicional

In [1]:
import numpy as np
def detectar_tipo_tarefa(y):
    y_array = np.array(y)

    if np.issubdtype(y_array.dtype, np.floating):
        return "regressao"

    n_classes = len(np.unique(y_array))
    if n_classes == 2:
        return "binaria"
    elif n_classes > 2:
        return "multiclasse"
    return "desconhecido"


In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def desempenho_tradicional_binario(y_true, y_pred, explain=False):
    acc = accuracy_score(y_true, y_pred)
    print(f"Acurácia:  {acc:.4f}")
    if explain:
        print("🔹 Acurácia: proporção de acertos no total de exemplos. Boa se > 0.80, mas pode enganar em dados desbalanceados.")

    prec = precision_score(y_true, y_pred)
    print(f"Precisão:  {prec:.4f}")
    if explain:
        print("🔹 Precisão: entre os que foram preditos como positivos, quantos realmente são. Boa se você quer evitar falsos positivos.")

    rec = recall_score(y_true, y_pred)
    print(f"Recall:    {rec:.4f}")
    if explain:
        print("🔹 Recall: entre os positivos reais, quantos foram detectados. Boa se você quer evitar falsos negativos.")

    f1 = f1_score(y_true, y_pred)
    print(f"F1-Score:  {f1:.4f}")
    if explain:
        print("🔹 F1-Score: média harmônica entre precisão e recall. Equilibra ambos quando há desbalanceamento.")
    return acc, prec, rec, f1

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def desempenho_tradicional_multiclasse(y_true, y_pred, explain=False):
    acc = accuracy_score(y_true, y_pred)
    print(f"Acurácia:  {acc:.4f}")
    if explain:
        print("🔹 Acurácia: proporção de acertos entre todas as classes. Boa se > 0.80, mas cuidado com desbalanceamento.")

    prec = precision_score(y_true, y_pred, average='macro')
    print(f"Precisão (macro): {prec:.4f}")
    if explain:
        print("🔹 Precisão: média da precisão de cada classe. Mostra se o modelo é justo com todas as classes.")

    rec = recall_score(y_true, y_pred, average='macro')
    print(f"Recall (macro):   {rec:.4f}")
    if explain:
        print("🔹 Recall: média do recall de cada classe. Mede cobertura média de cada classe verdadeira.")

    f1 = f1_score(y_true, y_pred, average='macro')
    print(f"F1-Score (macro): {f1:.4f}")
    if explain:
        print("🔹 F1-Score: média harmônica entre precisão e recall para todas as classes.")
    return acc, prec, rec, f1

In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def desempenho_tradicional_regressao(y_true, y_pred, explain=False):
    r2 = r2_score(y_true, y_pred)
    print(f"R2:  {r2:.4f}")
    if explain:
        print("🔹 R2: quão bem o modelo explica a variância. 1.0 é perfeito. Pode ser negativo se o modelo for ruim.")

    mse = mean_squared_error(y_true, y_pred)
    print(f"MSE: {mse:.4f}")
    if explain:
        print("🔹 MSE: erro quadrático médio. Penaliza mais os erros grandes. Quanto menor, melhor.")

    mae = mean_absolute_error(y_true, y_pred)
    print(f"MAE: {mae:.4f}")
    if explain:
        print("🔹 MAE: erro absoluto médio. Indica o erro médio em termos absolutos. Quanto menor, melhor.")
    return r2, mse, mae

In [5]:
def desempenho_tradicional(model, X_test, y_test, explain=False):
    tipo = detectar_tipo_tarefa(y_test)

    y_pred = model.predict(X_test)

    if tipo == "binaria":
        return desempenho_tradicional_binario(y_test, y_pred, explain)
    elif tipo == "multiclasse":
        return desempenho_tradicional_multiclasse(y_test, y_pred, explain)
    elif tipo == "regressao":
        return desempenho_tradicional_regressao(y_test, y_pred, explain)
    else:
        print("⚠️ Tipo de tarefa não reconhecido.")
        return None


## Explicabilidade

In [ ]:
import shap
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

def explain_model(model, X, max_display=10, nome=None, target_name=None):
    """
    Explica modelos de árvore com SHAP (suporta classificadores e regressões).
    
    Parâmetros:
        - model: modelo treinado (ex: DecisionTree, GBC, GBR)
        - X: dados de entrada (DataFrame ou array)
        - max_display: número máximo de features no gráfico
        - nome: título opcional a ser incluído nos gráficos
    """
    
    modelos_problema = isinstance(model, DecisionTreeClassifier)
    modelos_sem_problema = isinstance(model, (
        GradientBoostingClassifier,
        DecisionTreeRegressor,
        GradientBoostingRegressor
    ))

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)
    
    if not modelos_problema:
        if not modelos_sem_problema:
            print("⚠️ Tipo de modelo não identificado, tentando procedimento padrão")
            if shap_values.shape != X.shape:
                raise AssertionError(f"shap_values.shape = {shap_values.shape}, X.shape = {X.shape}")
                return
        
        print("🔍 Gerando explicabilidade global do modelo...")

        

        shap.summary_plot(shap_values, X, plot_type="bar", max_display=max_display, show=False)
        if nome:
            plt.title(f"{nome} — Importância global")
        plt.show()

        shap.summary_plot(shap_values, X, max_display=max_display, show=False)
        if nome:
            plt.title(f"{nome} — Distribuição dos impactos")
        plt.show()
    
    elif isinstance(model, DecisionTreeClassifier):
        print(f"shap_values.shape = {shap_values.shape}, X.shape = {X.shape}")
        class_names = model.classes_

        _, _, n_classes = shap_values.shape
        for i in range(n_classes):
            
            print(f"\n📊 {target_name}: {i}' — Explicação global")
            shap.summary_plot(shap_values[:, :, i], X, plot_type="bar", max_display=max_display, show=False)
            if nome:
                plt.title(f"{nome} — {target_name}: {i}' — Importância global")
            plt.show()

            print(f"📈 {target_name}: {i}' — Distribuição dos impactos")
            shap.summary_plot(shap_values[:, :, i], X, max_display=max_display, show=False)
            if nome:
                plt.title(f"{nome} — {target_name}: {i}' — Distribuição dos impactos")
            plt.show()


SyntaxError: unmatched ')' (4214720753.py, line 6)

In [ ]:
import shap
from scipy.special import expit
import matplotlib.pyplot as plt

def explain_individual(index, model, X_train):
    """
    🧠 Explicabilidade local com SHAP (versão waterfall).
    Mostra o impacto de cada feature + exibe a probabilidade prevista no gráfico.
    """
    # Obter valores SHAP
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_train)
    instance = X_train.iloc[[index]]

    # Corrigir acesso ao escalar
    expected_value = explainer.expected_value[0]
    fx = float(shap_values[index].sum() + expected_value)
    prob = float(expit(fx))

    # Construir explicação
    explanation = shap.Explanation(
        values=shap_values[index],
        base_values=expected_value,
        data=instance.values[0],
        feature_names=instance.columns
    )

    # Plot com título seguro (sem emoji)
    shap.plots.waterfall(explanation, show=False)
    plt.title(f"SHAP Waterfall — Instância {index} | Prob. prevista: {prob:.2f}", fontsize=12)
    plt.show()
